# Normalizer Performance Analysis

This notebook contains an analysis of the normalizer performance on the CIViC, MOA, and Clinvar data

## Import relevant packages

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
# from civicpy import civic as civicpy
# import plotly.express as px
import ndjson
import re
from dotenv import load_dotenv
import os

## Dictionaries to map variants to categories and record category counts

Bin variants to categories.

For variants with multiple associated types:  If the 2+ types have a subset relationship (eg frameshift; frameshift truncation), they are assigned to categories consistent with the superset type (frameshift).  If the types are disjoint (eg: Trancript Variant; Loss of Function Variant), they are assigned with the category most closely associated with the assayed data (Transcript Variant).  This assignment is done in the civic_category_bins dictionary.

In [2]:
civic_category_bins = {
    "Delins":"Sequence Variants",
    "Direct Tandem Duplication":"Sequence Variants",
    "Disruptive Inframe Deletion":"Sequence Variants",
    "Disruptive Inframe Insertion":"Sequence Variants",
    "Coding Sequence Variant":"Sequence Variants",
    "Conservative Inframe Deletion":"Sequence Variants",
    "Copy Number Variants":"Copy Number Variants",
    "Frameshift":"Sequence Variants",
    "Frameshift Truncation":"Sequence Variants",
    "Frameshift Variant":"Sequence Variants",
    "Frameshift Variant;Minus 1 Frameshift Variant":"Sequence Variants",
    "Inframe Deletion":"Sequence Variants",
    "Inframe Indel":"Sequence Variants",
    "Inframe Insertion":"Sequence Variants",
    "Intron Variant":"Region-Defined Variants",
    "Minus 1 Frameshift Variant":"Sequence Variants",
    "Minus 2 Frameshift Variant":"Sequence Variants",
    "Missense Variant":"Sequence Variants",
    "Non Conservative Missense Variant":"Sequence Variants",
    "Plus 1 Frameshift Variant":"Sequence Variants",
    "Region-Defined Variant":"Region-Defined Variants",
    "Regulatory Region Variant":"Region-Defined Variants",
    "Sequence Variants":"Sequence Variants",
    "Splice Acceptor Variant":"Region-Defined Variants",
    "Splice Donor Region Variant":"Region-Defined Variants",
    "Splice Donor Variant":"Region-Defined Variants",
    "Splicing Variant":"Other Variants",
    "Start Lost":"Sequence Variants",
    "Stop Gained":"Sequence Variants",
    "Stop Lost":"Sequence Variants",
    "Synonymous Variant":"Sequence Variants",
    "Transcript Amplification":"Copy Number Variants",
    "Transcript Fusion":"Fusion Variants",
    "3 Prime UTR Variant":"Region-Defined Variants",
    "Amino Acid Deletion;Inframe Deletion":"Sequence Variants",
    "Frameshift Truncation;Minus 2 Frameshift Variant":"Sequence Variants",
    "Frameshift Truncation;Plus 2 Frameshift Variant":"Sequence Variants",
    "Frameshift Variant;Delins":"Sequence Variants",
    "Inframe Insertion;Delins":"Sequence Variants",
    "Inframe Insertion;Inframe Deletion;Delins":"Sequence Variants",
    "Inframe Variant;Inframe Insertion;Inframe Deletion;Delins ":"Sequence Variants",
    "Minus 1 Frameshift Variant;Frameshift Truncation":"Sequence Variants",
    "Plus 1 Frameshift Variant;Frameshift Elongation":"Sequence Variants",
    "Plus 1 Frameshift Variant;Frameshift Truncation":"Sequence Variants",
    "Missense Variant;Gain Of Function Variant":"Sequence Variants", 
    "Missense Variant;Loss Of Function Variant":"Sequence Variants", 
    "Missense Variant;Loss Of Heterozygosity":"Sequence Variants", 
    "Missense Variant;Polymorphic Sequence Variant":"Sequence Variants", 
    "Missense Variant;Snp":"Sequence Variants", 
    "Missense Variant;Transcript Fusion":"Sequence Variants",
    "Stop Gained;Loss Of Function Variant":"Sequence Variants",
    "Stop Lost;Inframe Deletion":"Sequence Variants"
}



moa_category_bins = {
    "Copy Number Variants": "Copy Number Variants",
    "Expression Variants": "Expression Variants",
    "Other Variants": "Other Variants",
    "Rearrangement Variants": "Rearrangement Variants",
    "Sequence Variants": "Sequence Variants"
}



clinvar_category_bins = {
    "Complex":"Other Variants",
    "CompoundHeterozygote":"Genotype Variants",
    "Deletion":"Sequence Variants",
    "Diplotype":"Genotype Variants",
    "Distinct chromosomes":"Rearrangement Variants",
    "Duplication":"Sequence Variants",
    "Haplotype":"Sequence Variants",
    "Haplotype, single variant":"Sequence Variants",
    "Indel":"Sequence Variants",
    "Insertion":"Sequence Variants",
    "Inversion":"Sequence Variants",
    "Microsatellite":"Sequence Variants",
    "Phase unknown":"Other Variants",
    "Tandem duplication":"Sequence Variants",
    "Translocation":"Rearrangement Variants",
    "Variation":"Other Variants",
    "copy number gain":"Copy Number Variants",
    "copy number loss":"Copy Number Variants",
    "fusion":"Fusion Variants",
    "protein only":"Sequence Variants",
    "single nucleotide variant":"Sequence Variants"
}


# the values in this dictionary are lists of 4 integer values:
# [nomalized_count, unable_to_normalize_count, unsupported_count, total_count]
nomalized_count = 0
unable_to_normalize_count = 1
unsupported_count = 2
total_count = 3
percent_normalized = 4

category_counts = {
    "Copy Number Variants":[0,0,0,0,0.0],
    "Epigenetic Modification":[0,0,0,0,0.0],
    "Expression Variants":[0,0,0,0,0.0],
    "Fusion Variants":[0,0,0,0,0.0],
    "Gene Function Variants":[0,0,0,0,0.0],
    "Genotype Variants":[0,0,0,0,0.0],
    "Other Variants":[0,0,0,0,0.0],
    "Rearrangement Variants":[0,0,0,0,0.0],
    "Region-Defined Variants":[0,0,0,0,0.0],
    "Sequence Variants":[0,0,0,0,0.0]
}

## CIViC



In order to score the normalizer's performance on the CIViC data, some cleaning is required.

First we need to read in the data that was ostensibly supported, get rid of varients with multiple type labels, and assign variant types to as  many of the entries as possible that have a "Not provided" value for civic_variant_types.

Read in .csv of normalized variants in CIVIC

In [3]:
civic_normalized_df = pd.read_csv("../civic/variation_analysis/able_to_normalize_queries.csv",sep = "\t")
print(civic_normalized_df.shape)
civic_normalized_df.head()
type(civic_normalized_df)

(1876, 7)


pandas.core.frame.DataFrame

Prune columns and add new column to flag as normalized.

In [4]:
pruned_civic_normalized_df = civic_normalized_df[["variant_id","query","query_type","civic_variant_types"]]
pruned_civic_normalized_df.insert(4,"normalization_status","normalized")
pruned_civic_normalized_df.head()

,variant_id,query,query_type,civic_variant_types,normalization_status
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,Stop Lost,normalized
1,1988,NC_000003.11:g.10191649A>T,genomic,Stop Lost,normalized
2,2488,3-10191647-T-G,genomic,Stop Lost,normalized
3,1986,NC_000003.11:g.10191648G>T,genomic,Stop Lost,normalized
4,1987,NC_000003.11:g.10191649A>G,genomic,Stop Lost,normalized


Repeat process with the variants that were unable to be normalized.

In [5]:
civic_not_normalized_df = pd.read_csv("../civic/variation_analysis/unable_to_normalize_queries.csv",sep = "\t")
print(civic_not_normalized_df.shape)
civic_not_normalized_df.head()
type(civic_not_normalized_df)

(80, 8)


pandas.core.frame.DataFrame

In [6]:
pruned_civic_not_normalized_df = civic_not_normalized_df[["variant_id","query","query_type","civic_variant_types"]]
pruned_civic_not_normalized_df.insert(4,"normalization_status","not_normalized")
pruned_civic_not_normalized_df.head()

,variant_id,query,query_type,civic_variant_types,normalization_status
0,748,MLH1 *757L,protein,Stop Lost,not_normalized
1,3718,AR A748V,protein,Not provided,not_normalized
2,3725,AR A765T,protein,Not provided,not_normalized
3,4485,ERBB2 A775_G776ins YVMA,protein,Not provided,not_normalized
4,248,TERT C228T,protein,Regulatory Region Variant,not_normalized


Merge these dfs

In [7]:
frames = [pruned_civic_normalized_df, pruned_civic_not_normalized_df]
civic_supported_df = pd.concat(frames)
civic_supported_df.shape

(1956, 5)

Making all queries in all caps to make it easier to account of untyped variants later on.

In [8]:
civic_supported_df["query"] = civic_supported_df["query"].apply(str.upper)

Checking variant types.  The single largest types is "Not provided".  
Most of these look like amino acid substitutions.
Defining a regex to detect these variants and assign "Missense Varaint" type to these variants.

In [9]:
civic_supported_df["civic_variant_types"].value_counts(dropna=False)

civic_variant_types
Not provided                                                 816
Missense Variant                                             815
Stop Gained                                                   60
Frameshift Truncation                                         36
Transcript Amplification                                      36
Inframe Deletion                                              35
Minus 1 Frameshift Variant;Frameshift Truncation              21
Inframe Insertion                                             17
Synonymous Variant                                            13
Splice Donor Variant                                          13
Splice Acceptor Variant                                        9
Missense Variant;Gain Of Function Variant                      9
Frameshift Variant                                             7
Stop Lost                                                      6
Conservative Inframe Deletion                                  6
Delin

If a variant does not have an assigned variant type in civic, it is a protein query, and the query matches a regex pattern associated with variant flagstitutions (such as "PTEN A126D"), then I am re-classifying them as a "Missense Variant" instead.

In [10]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+[A-Z|*]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])

# civic_supported_df[0:20]
    # bool(re.match("\S+\s+[A-Z]+\d+[A-Z]","PTEN A126D"))

Doing so reduced the 816 untyped variants down to 70.
Checking the remaining weird variants.

In [11]:
untyped_variants = civic_supported_df[civic_supported_df["civic_variant_types"] == "Not provided"]
print(untyped_variants.head(20))

     variant_id                      query query_type civic_variant_types  \
11         3342          KRAS A11_G12INSGA    protein        Not provided   
65         4484     ERBB2 A775_G776INSIVMA    protein        Not provided   
66         2658     ERBB2 A775_G776INSYVMA    protein        Not provided   
67         4483     ERBB2 A775_G776INSYVMA    protein        Not provided   
74         3751  ARHGAP35 A865_L870DELINSV    protein        Not provided   
82         2655          MYB AMPLIFICATION    protein        Not provided   
110        1261         MDM2 AMPLIFICATION    protein        Not provided   
113        1276          SMO AMPLIFICATION    protein        Not provided   
116        1684        PSMD4 AMPLIFICATION    protein        Not provided   
118        2205         FLT4 AMPLIFICATION    protein        Not provided   
119        2240         TLK2 AMPLIFICATION    protein        Not provided   
120        2397         CRKL AMPLIFICATION    protein        Not provided   

Reassigning variants marked as {gene} Amplification as Transcript Amplification Variants

In [12]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+AMPLIFICATION", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Transcript Amplification", civic_supported_df["civic_variant_types"])


Reassigning amino acid insertions, delins, and deletions as "Missense Variant", including a couple of variants that have a random space before or after the sequence operation like "INS"

In [13]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+_+[A-Z]+\d+INS+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])


In [14]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+_+[A-Z]+\d+INS+\s+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])

In [15]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+-+\d+\s+INS+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])

In [16]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+_+[A-Z]+\d+DELINS+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])


In [17]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+_+[A-Z]+\d+DEL", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])


And assigning missense types to a handful of remaining variants that are non-standard names for genomic and protein sequence variants

In [18]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+P\.+[A-Z]+\d+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])


In [19]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+[A-Z]+\-+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "genomic") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])

In [20]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("NC_\d+\.+\d+:[A-Z]+\.+\d+[A-Z]+>+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "genomic") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])

In [21]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("NC_\d+\.+\d+:[A-Z]+\.+\d+_+\d+INS+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "genomic") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])

This last variant is a unique (to this db) nonstandard nomenclature for just some variant in a particular domain, so it is a region-defined variant.

In [22]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("DICER1 RNASE IIIB MUTATION", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Region-Defined Variant", civic_supported_df["civic_variant_types"])

Add category column to CIViC df.

In [23]:
civic_supported_df["category"] = civic_supported_df["civic_variant_types"].map(civic_category_bins)
civic_supported_df.tail()

,variant_id,query,query_type,civic_variant_types,normalization_status,variant flag,category
75,953,KIT Y553_K558DEL,protein,Inframe Deletion,not_normalized,False,Sequence Variants
76,1542,KIT Y553_W557DELYEVQW,protein,Inframe Deletion,not_normalized,False,Sequence Variants
77,1548,KIT Y570_L576DEL,protein,Inframe Deletion,not_normalized,False,Sequence Variants
78,1630,FLT3 Y591_V592INSVDFREYE,protein,Missense Variant,not_normalized,False,Sequence Variants
79,3724,AR Y763C,protein,Missense Variant,not_normalized,False,Sequence Variants


Split df by normalized/not_normalized flag

In [24]:
civic_normalized_df_cats = civic_supported_df[civic_supported_df["normalization_status"] == "normalized"]
civic_normalized_df_cats

,variant_id,query,query_type,civic_variant_types,normalization_status,variant flag,category
0,2489,NC_000003.11:G.10191648_10191649INSC,genomic,Stop Lost,normalized,False,Sequence Variants
1,1988,NC_000003.11:G.10191649A>T,genomic,Stop Lost,normalized,False,Sequence Variants
2,2488,3-10191647-T-G,genomic,Stop Lost,normalized,False,Sequence Variants
3,1986,NC_000003.11:G.10191648G>T,genomic,Stop Lost,normalized,False,Sequence Variants
4,1987,NC_000003.11:G.10191649A>G,genomic,Stop Lost,normalized,False,Sequence Variants
...,...,...,...,...,...,...,...
1871,3161,3-10183878-G-A,genomic,Missense Variant,normalized,False,Sequence Variants
1872,877,NC_000020.11:G.58903752C>T,genomic,Synonymous Variant,normalized,False,Sequence Variants
1873,731,NC_000003.11:G.37056036G>A,genomic,Splice Donor Variant,normalized,False,Region-Defined Variants
1874,3045,VHL P.F76DEL,protein,Missense Variant,normalized,False,Sequence Variants


In [25]:
civic_not_normalized_df_cats = civic_supported_df[civic_supported_df["normalization_status"] == "not_normalized"]
civic_not_normalized_df_cats

,variant_id,query,query_type,civic_variant_types,normalization_status,variant flag,category
0,748,MLH1 *757L,protein,Stop Lost,not_normalized,False,Sequence Variants
1,3718,AR A748V,protein,Missense Variant,not_normalized,False,Sequence Variants
2,3725,AR A765T,protein,Missense Variant,not_normalized,False,Sequence Variants
3,4485,ERBB2 A775_G776INS YVMA,protein,Missense Variant,not_normalized,False,Sequence Variants
4,248,TERT C228T,protein,Regulatory Region Variant,not_normalized,False,Region-Defined Variants
...,...,...,...,...,...,...,...
75,953,KIT Y553_K558DEL,protein,Inframe Deletion,not_normalized,False,Sequence Variants
76,1542,KIT Y553_W557DELYEVQW,protein,Inframe Deletion,not_normalized,False,Sequence Variants
77,1548,KIT Y570_L576DEL,protein,Inframe Deletion,not_normalized,False,Sequence Variants
78,1630,FLT3 Y591_V592INSVDFREYE,protein,Missense Variant,not_normalized,False,Sequence Variants


For each df, Get CIViC Variant counts by category and add to counts dictionary

In [26]:
civic_normalized_category_counts = civic_normalized_df_cats["category"].value_counts()
civic_normalized_category_counts.head()
for i in range(len(civic_normalized_category_counts)):
    category = civic_normalized_category_counts.index[i]
    count = civic_normalized_category_counts[i]
    print(category, count)
    category_counts[category][nomalized_count] += count
    category_counts[category][total_count] += count

for i in category_counts.items():
    print(i)


Sequence Variants 1788
Copy Number Variants 57
Region-Defined Variants 28
Fusion Variants 1
Other Variants 1
('Copy Number Variants', [57, 0, 0, 57, 0.0])
('Epigenetic Modification', [0, 0, 0, 0, 0.0])
('Expression Variants', [0, 0, 0, 0, 0.0])
('Fusion Variants', [1, 0, 0, 1, 0.0])
('Gene Function Variants', [0, 0, 0, 0, 0.0])
('Genotype Variants', [0, 0, 0, 0, 0.0])
('Other Variants', [1, 0, 0, 1, 0.0])
('Rearrangement Variants', [0, 0, 0, 0, 0.0])
('Region-Defined Variants', [28, 0, 0, 28, 0.0])
('Sequence Variants', [1788, 0, 0, 1788, 0.0])


In [27]:
civic_not_normalized_category_counts = civic_not_normalized_df_cats["category"].value_counts()
civic_not_normalized_category_counts.head()
for i in range(len(civic_not_normalized_category_counts)):
    category = civic_not_normalized_category_counts.index[i]
    count = civic_not_normalized_category_counts[i]
    print(category, count)
    category_counts[category][unable_to_normalize_count] += count
    category_counts[category][total_count] += count

for i in category_counts.items():
    print(i)

Sequence Variants 77
Region-Defined Variants 3
('Copy Number Variants', [57, 0, 0, 57, 0.0])
('Epigenetic Modification', [0, 0, 0, 0, 0.0])
('Expression Variants', [0, 0, 0, 0, 0.0])
('Fusion Variants', [1, 0, 0, 1, 0.0])
('Gene Function Variants', [0, 0, 0, 0, 0.0])
('Genotype Variants', [0, 0, 0, 0, 0.0])
('Other Variants', [1, 0, 0, 1, 0.0])
('Rearrangement Variants', [0, 0, 0, 0, 0.0])
('Region-Defined Variants', [28, 3, 0, 31, 0.0])
('Sequence Variants', [1788, 77, 0, 1865, 0.0])


Read in the csv for unsupported variants.  This data was already mapped to categories in civic_variant_analysis.  Therefore, we only need to import the data and perform the count on the category column.

In [28]:
not_supported_variants = pd.read_csv("../civic/variation_analysis/not_supported_variants.csv",sep = "\t")
print(not_supported_variants.shape)
not_supported_variants.head()

(1563, 6)


,variant_id,gene_name,variant_name,civic_variant_types,category,variant_accepted
0,4170,VHL,NaN,Not provided,Transcript Variants,False
1,4417,ALK,FBXO11::ALK,Not provided,Fusion Variants,False
2,4214,VHL,NaN,Not provided,Transcript Variants,False
3,4216,VHL,NaN,Not provided,Transcript Variants,False
4,4278,VHL,NaN,Not provided,Transcript Variants,False


Checking Counts.

In [29]:
not_supported_variants["category"].value_counts()

category
Transcript Variants        366
Fusion Variants            294
Expression Variants        287
Sequence Variants          133
Region Defined Variants    129
Rearrangement Variants     116
Gene Function Variants      91
Other Variants              83
Copy Number Variants        34
Genotype Variants           16
Epigenetic Modification     14
Name: count, dtype: int64

There are two small discrepencies here. First, there is a hyphen missing from "Region-Defined Variants" which will cause a key error.  Second, the variants labelled as "Transcript Variants" here should be binned under "Other Variants".  Fixing that now.

In [30]:
not_supported_variants["category"].replace("Region Defined Variants", "Region-Defined Variants", inplace=True)
not_supported_variants["category"].replace("Transcript Variants", "Other Variants", inplace=True)

In [31]:
not_supported_variants_category_counts = not_supported_variants["category"].value_counts()
not_supported_variants_category_counts.head()
for i in range(len(not_supported_variants_category_counts)):
    category = not_supported_variants_category_counts.index[i]
    count = not_supported_variants_category_counts[i]
    print(category, count)
    category_counts[category][unsupported_count] += count
    category_counts[category][total_count] += count

for i in category_counts.items():
    print(i)

Other Variants 449
Fusion Variants 294
Expression Variants 287
Sequence Variants 133
Region-Defined Variants 129
Rearrangement Variants 116
Gene Function Variants 91
Copy Number Variants 34
Genotype Variants 16
Epigenetic Modification 14
('Copy Number Variants', [57, 0, 34, 91, 0.0])
('Epigenetic Modification', [0, 0, 14, 14, 0.0])
('Expression Variants', [0, 0, 287, 287, 0.0])
('Fusion Variants', [1, 0, 294, 295, 0.0])
('Gene Function Variants', [0, 0, 91, 91, 0.0])
('Genotype Variants', [0, 0, 16, 16, 0.0])
('Other Variants', [1, 0, 449, 450, 0.0])
('Rearrangement Variants', [0, 0, 116, 116, 0.0])
('Region-Defined Variants', [28, 3, 129, 160, 0.0])
('Sequence Variants', [1788, 77, 133, 1998, 0.0])


## MOA

Read MOA .csv file for Normalized variants

In [32]:
moa_normalized_df = pd.read_csv("../moa/feature_analysis/able_to_normalize_queries.csv",sep = "\t")
print(moa_normalized_df.shape)
moa_normalized_df.head()
type(moa_normalized_df)

(181, 5)


pandas.core.frame.DataFrame

Get variant counts by category, update variant counts df 

In [33]:
moa_normalized_category_counts = moa_normalized_df["category"].value_counts(dropna=False)
moa_normalized_category_counts.head()
indeces = moa_normalized_category_counts.index
for i in range(len(moa_normalized_category_counts)):
    variant = moa_normalized_category_counts.index[i]
    count = moa_normalized_category_counts[i]
    print(variant, count)
    target_category = moa_category_bins[variant]
    # print(target_category)
    category_counts[target_category][nomalized_count] += count
    category_counts[target_category][total_count] += count

for i in category_counts.items():
    print(i)


Sequence Variants 149
Copy Number Variants 32
('Copy Number Variants', [89, 0, 34, 123, 0.0])
('Epigenetic Modification', [0, 0, 14, 14, 0.0])
('Expression Variants', [0, 0, 287, 287, 0.0])
('Fusion Variants', [1, 0, 294, 295, 0.0])
('Gene Function Variants', [0, 0, 91, 91, 0.0])
('Genotype Variants', [0, 0, 16, 16, 0.0])
('Other Variants', [1, 0, 449, 450, 0.0])
('Rearrangement Variants', [0, 0, 116, 116, 0.0])
('Region-Defined Variants', [28, 3, 129, 160, 0.0])
('Sequence Variants', [1937, 77, 133, 2147, 0.0])


Repeat same process for variants that were supported but failed to normalize.

In [34]:
moa_not_normalized_df = pd.read_csv("../moa/feature_analysis/unable_to_normalize_queries.csv",sep = "\t")
print(moa_not_normalized_df.shape)
moa_not_normalized_df.head()
type(moa_not_normalized_df)

(0, 7)


pandas.core.frame.DataFrame

In [35]:
moa__not_normalized_category_counts = moa_not_normalized_df["category"].value_counts(dropna=False)
moa__not_normalized_category_counts.head()
indeces = moa__not_normalized_category_counts.index
for i in range(len(moa__not_normalized_category_counts)):
    variant = moa__not_normalized_category_counts.index[i]
    count = moa__not_normalized_category_counts[i]
    print(variant, count)
    target_category = moa_category_bins[variant]
    # print(target_category)
    category_counts[target_category][unable_to_normalize_count] += count
    category_counts[target_category][total_count] += count

for i in category_counts.items():
    print(i)


('Copy Number Variants', [89, 0, 34, 123, 0.0])
('Epigenetic Modification', [0, 0, 14, 14, 0.0])
('Expression Variants', [0, 0, 287, 287, 0.0])
('Fusion Variants', [1, 0, 294, 295, 0.0])
('Gene Function Variants', [0, 0, 91, 91, 0.0])
('Genotype Variants', [0, 0, 16, 16, 0.0])
('Other Variants', [1, 0, 449, 450, 0.0])
('Rearrangement Variants', [0, 0, 116, 116, 0.0])
('Region-Defined Variants', [28, 3, 129, 160, 0.0])
('Sequence Variants', [1937, 77, 133, 2147, 0.0])


Repeat same process for variants that are unsupported.

In [36]:
moa_not_supported_df = pd.read_csv("../moa/feature_analysis/not_supported_variants.csv",sep = "\t")
print(moa_not_supported_df.shape)
print(moa_not_supported_df.head())
type(moa_not_supported_df)
print(moa_not_supported_df["category"].value_counts(dropna=False))

(249, 4)
   variant_id                        query moa_feature_type  \
0           1             BCR--ABL1 Fusion    rearrangement   
1          12                   ALK Fusion    rearrangement   
2          15                          ALK    rearrangement   
3          18            ALK Translocation    rearrangement   
4          21  BRD4 t(15;19) Translocation    rearrangement   

                 category  
0  Rearrangement Variants  
1  Rearrangement Variants  
2  Rearrangement Variants  
3  Rearrangement Variants  
4  Rearrangement Variants  
category
Sequence Variants         181
Rearrangement Variants     35
Copy Number Variants       17
Expression Variants        11
Other Variants              5
Name: count, dtype: int64


In [37]:
othervars = moa_not_supported_df[moa_not_supported_df["category"] == "Other Variants"]
# print(othervars.head())
type(othervars)
othervars.head

<bound method NDFrame.head of      variant_id                      query          moa_feature_type  \
210         781                   MSI-High  microsatellite_stability   
216         803                       High         mutational_burden   
217         805    High (>= 178 mutations)         mutational_burden   
218         806    High (>= 100 mutations)         mutational_burden   
219         808  High (>= 10 mutations/Mb)         mutational_burden   

           category  
210  Other Variants  
216  Other Variants  
217  Other Variants  
218  Other Variants  
219  Other Variants  >

In [38]:
moa__not_supported_category_counts = moa_not_supported_df["category"].value_counts(dropna=False)
moa__not_supported_category_counts.head()
indeces = moa__not_supported_category_counts.index
for i in range(len(moa__not_supported_category_counts)):
    variant = moa__not_supported_category_counts.index[i]
    count = moa__not_supported_category_counts[i]
    print(variant, count)
    target_category = moa_category_bins[variant]
    # print(target_category)
    category_counts[target_category][unsupported_count] += count
    category_counts[target_category][total_count] += count

for i in category_counts.items():
    print(i)


Sequence Variants 181
Rearrangement Variants 35
Copy Number Variants 17
Expression Variants 11
Other Variants 5
('Copy Number Variants', [89, 0, 51, 140, 0.0])
('Epigenetic Modification', [0, 0, 14, 14, 0.0])
('Expression Variants', [0, 0, 298, 298, 0.0])
('Fusion Variants', [1, 0, 294, 295, 0.0])
('Gene Function Variants', [0, 0, 91, 91, 0.0])
('Genotype Variants', [0, 0, 16, 16, 0.0])
('Other Variants', [1, 0, 454, 455, 0.0])
('Rearrangement Variants', [0, 0, 151, 151, 0.0])
('Region-Defined Variants', [28, 3, 129, 160, 0.0])
('Sequence Variants', [1937, 77, 314, 2328, 0.0])


## ClinVar

Read in the three clinvar csv files.

In [39]:
clinvar_normalized_df = pd.read_csv("../clinvar/clinvar_variation_analysis_output/variation_type_count_supported_df.csv")
print(clinvar_normalized_df.shape)
clinvar_normalized_df.head(20)

(20, 4)


,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count
0,0,single nucleotide variant,Canonical SPDI,1934692
1,1,Deletion,Canonical SPDI,92875
2,2,Duplication,Canonical SPDI,43185
3,3,copy number loss,Absolute copy count,26857
4,4,Microsatellite,Canonical SPDI,26678
5,5,copy number gain,Absolute copy count,25583
6,6,Deletion,Copy number change (cn loss|del and cn gain|dup),16249
7,7,Indel,Canonical SPDI,10986
8,8,Insertion,Canonical SPDI,8865
9,9,Duplication,Copy number change (cn loss|del and cn gain|dup),8119


In [40]:
clinvar_not_normalized_df = pd.read_csv("../clinvar/clinvar_variation_analysis_output/variation_type_count_supported_not_normalized_df.csv")
print(clinvar_not_normalized_df.shape)
clinvar_not_normalized_df.head(20)

(9, 4)


,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count
0,0,copy number loss,Absolute copy count,475
1,1,copy number gain,Absolute copy count,344
2,2,Deletion,Copy number change (cn loss|del and cn gain|dup),89
3,3,Duplication,Copy number change (cn loss|del and cn gain|dup),72
4,4,copy number loss,Copy number change (cn loss|del and cn gain|dup),31
5,5,copy number gain,Copy number change (cn loss|del and cn gain|dup),17
6,6,single nucleotide variant,Remaining valid hgvs alleles,11
7,7,Insertion,Remaining valid hgvs alleles,1
8,8,Variation,Remaining valid hgvs alleles,1


In [41]:
clinvar_not_supported_df = pd.read_csv("../clinvar/clinvar_variation_analysis_output/variation_type_count_not_supported_df.csv")
print(clinvar_not_supported_df.shape)
clinvar_not_supported_df.head(20)

(37, 4)


,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count
0,0,copy number gain,NCBI36 genomic only,2897
1,1,copy number loss,NCBI36 genomic only,1749
2,2,Deletion,No hgvs or location info,1236
3,3,Diplotype,Genotype/Haplotype,596
4,4,Haplotype,Genotype/Haplotype,565
5,5,Microsatellite,Invalid/unsupported hgvs,415
6,6,Deletion,Invalid/unsupported hgvs,413
7,7,Insertion,No hgvs or location info,382
8,8,Insertion,Invalid/unsupported hgvs,306
9,9,single nucleotide variant,No hgvs or location info,287


Add column and map variant types to categories.

In [42]:
clinvar_normalized_df["category"] = clinvar_normalized_df["in.variation_type"].map(clinvar_category_bins)
clinvar_normalized_df.head(20)

,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count,category
0,0,single nucleotide variant,Canonical SPDI,1934692,Sequence Variants
1,1,Deletion,Canonical SPDI,92875,Sequence Variants
2,2,Duplication,Canonical SPDI,43185,Sequence Variants
3,3,copy number loss,Absolute copy count,26857,Copy Number Variants
4,4,Microsatellite,Canonical SPDI,26678,Sequence Variants
5,5,copy number gain,Absolute copy count,25583,Copy Number Variants
6,6,Deletion,Copy number change (cn loss|del and cn gain|dup),16249,Sequence Variants
7,7,Indel,Canonical SPDI,10986,Sequence Variants
8,8,Insertion,Canonical SPDI,8865,Sequence Variants
9,9,Duplication,Copy number change (cn loss|del and cn gain|dup),8119,Sequence Variants


In [43]:
clinvar_not_normalized_df["category"] = clinvar_not_normalized_df["in.variation_type"].map(clinvar_category_bins)
clinvar_not_normalized_df.head(20)

,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count,category
0,0,copy number loss,Absolute copy count,475,Copy Number Variants
1,1,copy number gain,Absolute copy count,344,Copy Number Variants
2,2,Deletion,Copy number change (cn loss|del and cn gain|dup),89,Sequence Variants
3,3,Duplication,Copy number change (cn loss|del and cn gain|dup),72,Sequence Variants
4,4,copy number loss,Copy number change (cn loss|del and cn gain|dup),31,Copy Number Variants
5,5,copy number gain,Copy number change (cn loss|del and cn gain|dup),17,Copy Number Variants
6,6,single nucleotide variant,Remaining valid hgvs alleles,11,Sequence Variants
7,7,Insertion,Remaining valid hgvs alleles,1,Sequence Variants
8,8,Variation,Remaining valid hgvs alleles,1,Other Variants


In [44]:
clinvar_not_supported_df["category"] = clinvar_not_supported_df["in.variation_type"].map(clinvar_category_bins)
clinvar_not_supported_df.head(20)

,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count,category
0,0,copy number gain,NCBI36 genomic only,2897,Copy Number Variants
1,1,copy number loss,NCBI36 genomic only,1749,Copy Number Variants
2,2,Deletion,No hgvs or location info,1236,Sequence Variants
3,3,Diplotype,Genotype/Haplotype,596,Genotype Variants
4,4,Haplotype,Genotype/Haplotype,565,Sequence Variants
5,5,Microsatellite,Invalid/unsupported hgvs,415,Sequence Variants
6,6,Deletion,Invalid/unsupported hgvs,413,Sequence Variants
7,7,Insertion,No hgvs or location info,382,Sequence Variants
8,8,Insertion,Invalid/unsupported hgvs,306,Sequence Variants
9,9,single nucleotide variant,No hgvs or location info,287,Sequence Variants


Due to the structure of the data and the way that the original analysis developped, some but not all CNVs per the in.variation_type were annotated in the in.vrs_xform_plan.policy column as "Copy number change (cn loss|del and cn gain|dup)", "Absolute copy count", or "Min/max copy count range not supported".  However, some of the Copy number Gain/Loss variants were did not get binned as CNVs per the in.vrs_xform_plan.policy.  Therefore, we need to mark those variants in the union of the following two sets as being in the category of Copy Number Variants:

Variants with in.variant_type ==
1. copy number loss
2. copy number gain

Variants with in.vrs_xform_plan.policy == 
1. Copy number change (cn loss|del and cn gain|dup)
2. Absolute copy count
3. Min/max copy count range not supported

Above we already caught the first set of variants. Now we must go back through each df one more time and map the variants we missed per in.vrs_xform_plan.policy values to the category of Copy Number Variants.

In [45]:
cnv_per_policy = ["Copy number change (cn loss|del and cn gain|dup)","Absolute copy count","Min/max copy count range not supported","Copy number change (cn loss|del and cn gain|dup)"]

In [46]:
clinvar_normalized_df.loc[
    clinvar_normalized_df["in.vrs_xform_plan.policy"].isin(cnv_per_policy),
      "category"
      ] = "Copy Number Variants"


In [47]:
clinvar_normalized_df

,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count,category
0,0,single nucleotide variant,Canonical SPDI,1934692,Sequence Variants
1,1,Deletion,Canonical SPDI,92875,Sequence Variants
2,2,Duplication,Canonical SPDI,43185,Sequence Variants
3,3,copy number loss,Absolute copy count,26857,Copy Number Variants
4,4,Microsatellite,Canonical SPDI,26678,Sequence Variants
5,5,copy number gain,Absolute copy count,25583,Copy Number Variants
6,6,Deletion,Copy number change (cn loss|del and cn gain|dup),16249,Copy Number Variants
7,7,Indel,Canonical SPDI,10986,Sequence Variants
8,8,Insertion,Canonical SPDI,8865,Sequence Variants
9,9,Duplication,Copy number change (cn loss|del and cn gain|dup),8119,Copy Number Variants


In [48]:
clinvar_not_normalized_df

,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count,category
0,0,copy number loss,Absolute copy count,475,Copy Number Variants
1,1,copy number gain,Absolute copy count,344,Copy Number Variants
2,2,Deletion,Copy number change (cn loss|del and cn gain|dup),89,Sequence Variants
3,3,Duplication,Copy number change (cn loss|del and cn gain|dup),72,Sequence Variants
4,4,copy number loss,Copy number change (cn loss|del and cn gain|dup),31,Copy Number Variants
5,5,copy number gain,Copy number change (cn loss|del and cn gain|dup),17,Copy Number Variants
6,6,single nucleotide variant,Remaining valid hgvs alleles,11,Sequence Variants
7,7,Insertion,Remaining valid hgvs alleles,1,Sequence Variants
8,8,Variation,Remaining valid hgvs alleles,1,Other Variants


In [49]:
clinvar_not_normalized_df.loc[
    clinvar_not_normalized_df["in.vrs_xform_plan.policy"].isin(cnv_per_policy),
      "category"
      ] = "Copy Number Variants"


clinvar_not_normalized_df

,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count,category
0,0,copy number loss,Absolute copy count,475,Copy Number Variants
1,1,copy number gain,Absolute copy count,344,Copy Number Variants
2,2,Deletion,Copy number change (cn loss|del and cn gain|dup),89,Copy Number Variants
3,3,Duplication,Copy number change (cn loss|del and cn gain|dup),72,Copy Number Variants
4,4,copy number loss,Copy number change (cn loss|del and cn gain|dup),31,Copy Number Variants
5,5,copy number gain,Copy number change (cn loss|del and cn gain|dup),17,Copy Number Variants
6,6,single nucleotide variant,Remaining valid hgvs alleles,11,Sequence Variants
7,7,Insertion,Remaining valid hgvs alleles,1,Sequence Variants
8,8,Variation,Remaining valid hgvs alleles,1,Other Variants


In [50]:
clinvar_not_supported_df

,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count,category
0,0,copy number gain,NCBI36 genomic only,2897,Copy Number Variants
1,1,copy number loss,NCBI36 genomic only,1749,Copy Number Variants
2,2,Deletion,No hgvs or location info,1236,Sequence Variants
3,3,Diplotype,Genotype/Haplotype,596,Genotype Variants
4,4,Haplotype,Genotype/Haplotype,565,Sequence Variants
5,5,Microsatellite,Invalid/unsupported hgvs,415,Sequence Variants
6,6,Deletion,Invalid/unsupported hgvs,413,Sequence Variants
7,7,Insertion,No hgvs or location info,382,Sequence Variants
8,8,Insertion,Invalid/unsupported hgvs,306,Sequence Variants
9,9,single nucleotide variant,No hgvs or location info,287,Sequence Variants


In [51]:
clinvar_not_supported_df.loc[
    clinvar_not_supported_df["in.vrs_xform_plan.policy"].isin(cnv_per_policy),
      "category"
      ] = "Copy Number Variants"

clinvar_not_supported_df

,Unnamed: 0,in.variation_type,in.vrs_xform_plan.policy,count,category
0,0,copy number gain,NCBI36 genomic only,2897,Copy Number Variants
1,1,copy number loss,NCBI36 genomic only,1749,Copy Number Variants
2,2,Deletion,No hgvs or location info,1236,Sequence Variants
3,3,Diplotype,Genotype/Haplotype,596,Genotype Variants
4,4,Haplotype,Genotype/Haplotype,565,Sequence Variants
5,5,Microsatellite,Invalid/unsupported hgvs,415,Sequence Variants
6,6,Deletion,Invalid/unsupported hgvs,413,Sequence Variants
7,7,Insertion,No hgvs or location info,382,Sequence Variants
8,8,Insertion,Invalid/unsupported hgvs,306,Sequence Variants
9,9,single nucleotide variant,No hgvs or location info,287,Sequence Variants


Get counts from the three dfs.

In [52]:
category_counts

{'Copy Number Variants': [89, 0, 51, 140, 0.0],
 'Epigenetic Modification': [0, 0, 14, 14, 0.0],
 'Expression Variants': [0, 0, 298, 298, 0.0],
 'Fusion Variants': [1, 0, 294, 295, 0.0],
 'Gene Function Variants': [0, 0, 91, 91, 0.0],
 'Genotype Variants': [0, 0, 16, 16, 0.0],
 'Other Variants': [1, 0, 454, 455, 0.0],
 'Rearrangement Variants': [0, 0, 151, 151, 0.0],
 'Region-Defined Variants': [28, 3, 129, 160, 0.0],
 'Sequence Variants': [1937, 77, 314, 2328, 0.0]}

In [53]:
for i in category_counts.keys():
    subdf = clinvar_normalized_df[clinvar_normalized_df["category"] == i]
    if len(subdf) > 0:
        category = i
        count = subdf["count"].sum()
        print(category, count)
        category_counts[category][nomalized_count] += count
        category_counts[category][total_count] += count


for i in category_counts.items():
    print(i)


Copy Number Variants 79339
Other Variants 338
Sequence Variants 2119259
('Copy Number Variants', [79428, 0, 51, 79479, 0.0])
('Epigenetic Modification', [0, 0, 14, 14, 0.0])
('Expression Variants', [0, 0, 298, 298, 0.0])
('Fusion Variants', [1, 0, 294, 295, 0.0])
('Gene Function Variants', [0, 0, 91, 91, 0.0])
('Genotype Variants', [0, 0, 16, 16, 0.0])
('Other Variants', [339, 0, 454, 793, 0.0])
('Rearrangement Variants', [0, 0, 151, 151, 0.0])
('Region-Defined Variants', [28, 3, 129, 160, 0.0])
('Sequence Variants', [2121196, 77, 314, 2121587, 0.0])


In [54]:
for i in category_counts.keys():
    subdf = clinvar_not_normalized_df[clinvar_not_normalized_df["category"] == i]
    if len(subdf) > 0:
        category = i
        count = subdf["count"].sum()
        print(category, count)
        category_counts[category][unable_to_normalize_count] += count
        category_counts[category][total_count] += count


for i in category_counts.items():
    print(i)


Copy Number Variants 1028
Other Variants 1
Sequence Variants 12
('Copy Number Variants', [79428, 1028, 51, 80507, 0.0])
('Epigenetic Modification', [0, 0, 14, 14, 0.0])
('Expression Variants', [0, 0, 298, 298, 0.0])
('Fusion Variants', [1, 0, 294, 295, 0.0])
('Gene Function Variants', [0, 0, 91, 91, 0.0])
('Genotype Variants', [0, 0, 16, 16, 0.0])
('Other Variants', [339, 1, 454, 794, 0.0])
('Rearrangement Variants', [0, 0, 151, 151, 0.0])
('Region-Defined Variants', [28, 3, 129, 160, 0.0])
('Sequence Variants', [2121196, 89, 314, 2121599, 0.0])


In [55]:
for i in category_counts.keys():
    subdf = clinvar_not_supported_df[clinvar_not_supported_df["category"] == i]
    if len(subdf) > 0:
        category = i
        count = subdf["count"].sum()
        print(category, count)
        category_counts[category][unsupported_count] += count
        category_counts[category][total_count] += count


for i in category_counts.items():
    print(i)


Copy Number Variants 4672
Fusion Variants 6
Genotype Variants 845
Other Variants 293
Rearrangement Variants 274
Sequence Variants 4560
('Copy Number Variants', [79428, 1028, 4723, 85179, 0.0])
('Epigenetic Modification', [0, 0, 14, 14, 0.0])
('Expression Variants', [0, 0, 298, 298, 0.0])
('Fusion Variants', [1, 0, 300, 301, 0.0])
('Gene Function Variants', [0, 0, 91, 91, 0.0])
('Genotype Variants', [0, 0, 861, 861, 0.0])
('Other Variants', [339, 1, 747, 1087, 0.0])
('Rearrangement Variants', [0, 0, 425, 425, 0.0])
('Region-Defined Variants', [28, 3, 129, 160, 0.0])
('Sequence Variants', [2121196, 89, 4874, 2126159, 0.0])


# Computing Coverage

For the purposes of making the table, computing the percent of all variants normalized in each category.

In [56]:
for i in category_counts.keys():
    # print(i)
    # print(category_counts[i][nomalized_count])
    # print(type(category_counts[i][nomalized_count]))
    normalized = category_counts[i][nomalized_count]
    total = category_counts[i][total_count]
    # print(normalized, total)
    # print(normalized/total)
    percent_covered = normalized/total
    category_counts[i][percent_normalized] = "%.4f" % percent_covered

for i in category_counts.items():
    print(i)
    

('Copy Number Variants', [79428, 1028, 4723, 85179, '0.9325'])
('Epigenetic Modification', [0, 0, 14, 14, '0.0000'])
('Expression Variants', [0, 0, 298, 298, '0.0000'])
('Fusion Variants', [1, 0, 300, 301, '0.0033'])
('Gene Function Variants', [0, 0, 91, 91, '0.0000'])
('Genotype Variants', [0, 0, 861, 861, '0.0000'])
('Other Variants', [339, 1, 747, 1087, '0.3119'])
('Rearrangement Variants', [0, 0, 425, 425, '0.0000'])
('Region-Defined Variants', [28, 3, 129, 160, '0.1750'])
('Sequence Variants', [2121196, 89, 4874, 2126159, '0.9977'])


# Generating Table

In [130]:
from plotly.validators.scatter.marker import SymbolValidator
print(SymbolValidator().values)
print("\u23FA")
print("\u25EF")
print("\u2B24")
print('\u25CB')
print("\u25CF")
print("\u26AB")

[0, '0', 'circle', 100, '100', 'circle-open', 200, '200', 'circle-dot', 300, '300', 'circle-open-dot', 1, '1', 'square', 101, '101', 'square-open', 201, '201', 'square-dot', 301, '301', 'square-open-dot', 2, '2', 'diamond', 102, '102', 'diamond-open', 202, '202', 'diamond-dot', 302, '302', 'diamond-open-dot', 3, '3', 'cross', 103, '103', 'cross-open', 203, '203', 'cross-dot', 303, '303', 'cross-open-dot', 4, '4', 'x', 104, '104', 'x-open', 204, '204', 'x-dot', 304, '304', 'x-open-dot', 5, '5', 'triangle-up', 105, '105', 'triangle-up-open', 205, '205', 'triangle-up-dot', 305, '305', 'triangle-up-open-dot', 6, '6', 'triangle-down', 106, '106', 'triangle-down-open', 206, '206', 'triangle-down-dot', 306, '306', 'triangle-down-open-dot', 7, '7', 'triangle-left', 107, '107', 'triangle-left-open', 207, '207', 'triangle-left-dot', 307, '307', 'triangle-left-open-dot', 8, '8', 'triangle-right', 108, '108', 'triangle-right-open', 208, '208', 'triangle-right-dot', 308, '308', 'triangle-right-open

In [164]:
round(float(category_counts["Sequence Variants"][percent_normalized])*100,2)

99.77

In [166]:
import plotly.graph_objects as go

import pandas as pd
base_colors = ['rgb(49, 130, 189)','rgb(239, 243, 255)', 'rgb(189, 215, 231)', 'rgb(107, 174, 214)',
           'white']
core_field = "\u2B24"
optional_field = "◯"
 

colors = ['rgb(49, 130, 189)','white', 'white', 'white',
           'white', 'rgb(49, 130, 189)', 'white', 'white', 'rgb(189, 215, 231)','rgb(107, 174, 214)']
data = {'variant_category' : ["Sequence Variants", "Genotype Variants", "Fusion Variants", "Rearrangement Variants", "Epigenetic Variants", "Copy Number Variants", "Expression Variants",  "Gene Function Variants", "Region-Defined Variants", "Other Variants"],
        'counts' : [category_counts["Sequence Variants"][total_count], category_counts["Genotype Variants"][total_count], category_counts["Fusion Variants"][total_count], category_counts["Rearrangement Variants"][total_count], category_counts["Epigenetic Modification"][total_count], category_counts["Copy Number Variants"][total_count], category_counts["Expression Variants"][total_count], category_counts["Gene Function Variants"][total_count], category_counts["Region-Defined Variants"][total_count], category_counts["Other Variants"][total_count]],
        'percent_normalized' : [round(float(category_counts["Sequence Variants"][percent_normalized])*100,2), round(float(category_counts["Genotype Variants"][percent_normalized])*100,2), round(float(category_counts["Fusion Variants"][percent_normalized])*100,2), round(float(category_counts["Rearrangement Variants"][percent_normalized])*100,2), round(float(category_counts["Epigenetic Modification"][percent_normalized])*100,2), round(float(category_counts["Copy Number Variants"][percent_normalized])*100,2), round(float(category_counts["Expression Variants"][percent_normalized])*100,2), round(float(category_counts["Gene Function Variants"][percent_normalized])*100,2), round(float(category_counts["Region-Defined Variants"][percent_normalized])*100,2), round(float(category_counts["Other Variants"][percent_normalized])*100,2)],
        'delta_sequence' : [core_field, core_field, "", "", "", "", "", "", "", optional_field],
        'delta_location' : [optional_field, optional_field, core_field, core_field, "", "", "", "", "", ""],
        'delta_frame' : [optional_field, optional_field, "", "", "", "", "", "", "", optional_field],
        'delta_quantity' : [optional_field, optional_field, "", "", core_field, core_field, optional_field, "", "", optional_field],
        'delta_function' : [optional_field, optional_field, "", "", optional_field, optional_field, core_field, core_field, "", optional_field],
        'region_specificity' : [optional_field, optional_field, optional_field, optional_field, optional_field, optional_field, optional_field, optional_field, core_field, optional_field],
        'shading' : colors
         }
df = pd.DataFrame(data)

fig = go.Figure(data=[go.Table(
  header=dict(
    values=["<b>Variant Category</b>", "<b>Count</b>", "<b>% Normalized</b>", "<b>Δ Sequence</b>", "<b>Δ Location</b>", "<b>Δ Frame</b>", "<b>Δ Quantity</b>", "<b>Δ Function</b>", "<b>Region Specificity</b>"],
    line_color='black', fill_color='white',
    align='center', font=dict(color='black', size=12)
  ),
  cells=dict(
    values=[df.variant_category, df.counts, df.percent_normalized, df.delta_sequence, df.delta_location, df.delta_frame, df.delta_quantity, df.delta_function, df.region_specificity],
    line_color=["black"], fill_color=[df.shading],
    align='center', font=dict(color='black', size=11)
  ))
])
fig.show()